In [14]:
import json
from json import dumps
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit, explode, from_json, col, current_date,current_timestamp, lit

from pyspark.sql.types import StringType, StructType, StructField, ArrayType
import datetime 
import requests
import urllib3
from datetime import datetime,date
import pytz
import io
import glob
import os
import boto3
from pyspark.sql import SparkSession
import pyspark

In [21]:
import boto3
s3 = boto3.client('s3', endpoint_url='http://minio:9000',
                  aws_access_key_id=AWS_ACCESS_KEY,
                  aws_secret_access_key=AWS_SECRET_KEY)
s3.list_buckets()

{'ResponseMetadata': {'RequestId': '186D06831F4A37C7',
  'HostId': 'dd9025bab4ad464b049177c95eb6ebf374d3b3fd1af9251148b658df7ac2e3e8',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'accept-ranges': 'bytes',
   'content-length': '540',
   'content-type': 'application/xml',
   'server': 'MinIO',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'vary': 'Origin, Accept-Encoding',
   'x-amz-bucket-region': 'us-east-1',
   'x-amz-id-2': 'dd9025bab4ad464b049177c95eb6ebf374d3b3fd1af9251148b658df7ac2e3e8',
   'x-amz-request-id': '186D06831F4A37C7',
   'x-content-type-options': 'nosniff',
   'x-ratelimit-limit': '3111',
   'x-ratelimit-remaining': '3111',
   'x-xss-protection': '1; mode=block',
   'date': 'Fri, 10 Oct 2025 04:23:09 GMT'},
  'RetryAttempts': 0},
 'Buckets': [{'Name': 'bronze',
   'CreationDate': datetime.datetime(2025, 10, 10, 3, 5, 6, 762000, tzinfo=tzlocal())},
  {'Name': 'gold',
   'CreationDate': datetime.datetime(2025, 10, 10, 3, 5, 16, 197000, tzinfo=t

In [28]:
CATALOG_URI = "http://nessie:19120/api/v1"  # Nessie Server URI
WAREHOUSE = "s3://silver/"               # Minio Address to Write to
STORAGE_URI = "http://minio:9000"      
AWS_ACCESS_KEY='admin'
AWS_SECRET_KEY='password'

In [29]:
minio_client = boto3.client(
's3', endpoint_url='http://minio:9000',
                  aws_access_key_id=AWS_ACCESS_KEY,
                  aws_secret_access_key=AWS_SECRET_KEY)

In [30]:
conf = (
    pyspark.SparkConf()
        .setAppName('combined_spark_app')

        # Paquetes compatibles con Spark 3.5.2 y Scala 2.12
        .set('spark.jars.packages', ','.join([
            'org.postgresql:postgresql:42.7.3',
            'org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.5.0',
            'org.projectnessie.nessie-integrations:nessie-spark-extensions-3.5_2.12:0.77.1',
            'software.amazon.awssdk:bundle:2.24.8',
            'software.amazon.awssdk:url-connection-client:2.24.8',
            'org.apache.hadoop:hadoop-aws:3.3.4'  # ✅ actualizado para evitar conflictos
        ]))

        # Extensiones de Iceberg y Nessie
        .set('spark.sql.extensions', ','.join([
            'org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions',
            'org.projectnessie.spark.extensions.NessieSparkSessionExtensions'
        ]))

        # Catálogo Nessie para Iceberg
        .set('spark.sql.catalog.nessie', 'org.apache.iceberg.spark.SparkCatalog')
        .set('spark.sql.catalog.nessie.uri', CATALOG_URI)
        .set('spark.sql.catalog.nessie.ref', 'main')
        .set('spark.sql.catalog.nessie.authentication.type', 'NONE')
        .set('spark.sql.catalog.nessie.catalog-impl', 'org.apache.iceberg.nessie.NessieCatalog')

        # Configuración de almacenamiento en S3 (MinIO)
        .set('spark.sql.catalog.nessie.s3.endpoint', STORAGE_URI)
        .set('spark.sql.catalog.nessie.warehouse', WAREHOUSE)
        .set('spark.sql.catalog.nessie.io-impl', 'org.apache.iceberg.aws.s3.S3FileIO')

        # Acceso directo a S3 desde Spark
        .set('spark.hadoop.fs.s3a.endpoint', STORAGE_URI)
        .set('spark.hadoop.fs.s3a.access.key', AWS_ACCESS_KEY)
        .set('spark.hadoop.fs.s3a.secret.key', AWS_SECRET_KEY)
        .set('spark.hadoop.fs.s3a.path.style.access', 'true')
        .set('spark.hadoop.fs.s3a.aws.credentials.provider',
             'org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider')
)

spark = SparkSession.builder.config(conf=conf).getOrCreate()

In [6]:
spark = SparkSession.builder \
    .config(conf=conf) \
    .getOrCreate()
print("Spark Session Started")

Spark Session Started


In [15]:
votes_2021 = spark.read.parquet("s3a://bronze/votes/_2021/1760067419.8580947.e8bff62c90.parquet")
votes_2021 = votes_2021.withColumn("load_date", current_timestamp())

In [16]:
votes_2022 = spark.read.parquet("s3a://bronze/votes/_2022/1760067470.818231.d617815791.parquet")
votes_2022 = votes_2022.withColumn("load_date", current_timestamp())

In [17]:
spark.sql("CREATE NAMESPACE IF NOT EXISTS nessie.bronze").show()

++
||
++
++



In [25]:
votes_2021.writeTo("nessie.bronze.votes_2021").createOrReplace()

Py4JJavaError: An error occurred while calling o105.createOrReplace.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 11 in stage 9.0 failed 1 times, most recent failure: Lost task 11.0 in stage 9.0 (TID 53) (15b1df01654f executor driver): software.amazon.awssdk.core.exception.SdkClientException: Received an UnknownHostException when attempting to interact with a service. See cause for the exact endpoint that is failing to resolve. If this is happening on an endpoint that previously worked, there may be a network connectivity issue or your DNS cache could be storing endpoints for too long.
	at software.amazon.awssdk.core.exception.SdkClientException$BuilderImpl.build(SdkClientException.java:111)
	at software.amazon.awssdk.awscore.interceptor.HelpfulUnknownHostExceptionInterceptor.modifyException(HelpfulUnknownHostExceptionInterceptor.java:59)
	at software.amazon.awssdk.core.interceptor.ExecutionInterceptorChain.modifyException(ExecutionInterceptorChain.java:181

In [26]:
votes_2021.printSchema()

root
 |-- id: long (nullable = true)
 |-- post_id: long (nullable = true)
 |-- vote_type_id: long (nullable = true)
 |-- creation_date: timestamp (nullable = true)
 |-- user_id: long (nullable = true)
 |-- bounty_amount: decimal(20,0) (nullable = true)
 |-- load_date: timestamp (nullable = false)



In [ ]:
spark.stop()